In [14]:
import os
import random
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd
import math
from requests_html import HTMLSession

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ale\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ale\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ale\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
# Escoge 100 archivos al azar del directorio de reviews
reviews_folder_path = "reviews/"  
all_reviews_files = os.listdir(reviews_folder_path)
random_files = random.sample(all_reviews_files, 100)
print(random_files)

['7395_2.txt', '5043_3.txt', '4990_1.txt', '7261_3.txt', '4726_1.txt', '5489_1.txt', '4031_4.txt', '5892_2.txt', '8965_1.txt', '8198_4.txt', '1926_3.txt', '4903_3.txt', '854_1.txt', '5343_1.txt', '7839_4.txt', '3846_2.txt', '9276_1.txt', '3738_2.txt', '9642_4.txt', '2373_3.txt', '1723_1.txt', '8495_3.txt', '2205_2.txt', '9297_1.txt', '10905_3.txt', '7321_2.txt', '11291_1.txt', '12364_1.txt', '12194_3.txt', '10602_4.txt', '12289_1.txt', '2465_2.txt', '5621_4.txt', '4731_4.txt', '11513_3.txt', '12419_1.txt', '1655_2.txt', '8063_2.txt', '5891_3.txt', '7748_2.txt', '148_2.txt', '3923_2.txt', '10429_1.txt', '737_1.txt', '4737_3.txt', '7387_1.txt', '9302_1.txt', '410_4.txt', '11384_2.txt', '5159_2.txt', '1167_4.txt', '5291_4.txt', '1071_4.txt', '116_1.txt', '809_3.txt', '2343_4.txt', '1411_4.txt', '11264_1.txt', '9445_1.txt', '12095_3.txt', '7269_3.txt', '471_4.txt', '3560_2.txt', '1531_1.txt', '9066_4.txt', '7168_4.txt', '4931_1.txt', '5767_3.txt', '7998_1.txt', '8605_4.txt', '4664_3.txt', 

In [16]:
with open("reviews_url.txt", "r") as file:
    lines = file.readlines()

urls_random_files = []
for file in random_files:
    first_number = file.split("_")[0]
    id = lines[int(first_number)].replace("/usercomments\n", "")
    urls_random_files.append(id)

In [17]:
titles = []
session = HTMLSession()

for url in urls_random_files:
    respone = session.get(url)
    page_html = respone.html
    title= page_html.find('title')[0].text
    final_title = title.split("-")[0]
    titles.append(final_title)

titles

['Tim Watcher (Short 2003) ',
 'Zi hu die (2003) ',
 'Walking Tall: Lone Justice (Video 2007) ',
 'A Town Called Bastard (1971) ',
 'Shooting Livien (2005) ',
 'My Favorite Martian (1999) ',
 'The Lost Missile (1958) ',
 'Chakushin ari (2003) ',
 'Cube Zero (2004) ',
 'Alien Hunter (2003) ',
 'El Grinch (2000) ',
 'Ci qing (2007) ',
 'El resplandor (1980) ',
 'S Club Seeing Double (2003) ',
 'Shooting Dogs (2005) ',
 'Of Human Bondage (1934) ',
 'Omen IV: The Awakening (TV Movie 1991) ',
 'Made Men (1999) ',
 'Biohazard (1985) ',
 "Freddy's Dead: The Final Nightmare (1991) ",
 'The Crater Lake Monster (1977) ',
 'Conceiving Ada (1997) ',
 'Hammerhead (TV Movie 2005) ',
 'Executive Power (Video 1997) ',
 'The Animal (2001) ',
 'Acting on Impulse (1993) ',
 "Sharks' Treasure (1975) ",
 'Loonatics Unleashed (TV Series 2005–2007) ',
 'Ricordati di me (2003) ',
 'Universal Soldier: The Return (1999) ',
 'La casa 4 (Witchcraft) (1988) ',
 'Second to Die (2002) ',
 "The God Who Wasn't There (

In [18]:
# Preprocesamiento de los 100 archivos escogidos
# Incluye eliminar numeros, tildes y caracteres adiciones
# Lematizacion

lemmatized_directory = 'lemmatized_reviews/'
os.makedirs(lemmatized_directory, exist_ok=True)

lemmatizer = WordNetLemmatizer()
    
for file in random_files:
    path = os.path.join(reviews_folder_path, file)
        
    with open(path, 'r') as archivo:
        text = archivo.read()
        text=text.lower()
        text=text.replace("< br / >","")
        text=text.replace("<br />","")
        text=text.replace("´", "'")
        text = text.translate(str.maketrans('', '', string.punctuation))
        tokens = nltk.word_tokenize(text)
        stop_words = set(stopwords.words('english'))
        filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
        filtered = [word for word in filtered_tokens if word.isalpha() and word not in stop_words]
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered]
        lemmatized_text = ' '.join(lemmatized_tokens)
            
        # Create a new file with lemmatized content
        lemmatized_path = os.path.join(lemmatized_directory, file)
        with open(lemmatized_path, "w") as output_file:
            output_file.write(lemmatized_text)

print("Lemmatized files have been created in the 'lemmatized_reviews' directory.")

Lemmatized files have been created in the 'lemmatized_reviews' directory.


In [19]:
documents = []

for filename in random_files:
    file_number = filename.split("_")[1].split('.')[0]

    with open(os.path.join(lemmatized_directory, filename), 'r') as input_file:
        file_content = input_file.read()
        documents.append(file_content)

In [20]:
# Frecuencia por documento vectorizado 
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)
term_frequency = X.toarray()
document_frequency = np.sum(X > 0, axis=0)
document_frequency = document_frequency.tolist()[0]
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
tfidf_vectors = tfidf_matrix.toarray()

In [21]:
feature_names = tfidf_vectorizer.get_feature_names_out()

word_freq_dicts = {}

for random_files_index, random_file in enumerate(random_files):
    name_doc = "Doc"+ str(random_files_index) + "_tf"
    word_freq_per_doc_dict = {}
    for word_index, word in enumerate(feature_names):
        word_freq_per_doc_dict[word] = int(term_frequency[random_files_index][word_index])
    
    word_freq_dicts[random_file] = word_freq_per_doc_dict

freq_df = pd.DataFrame(word_freq_dicts)
freq_df

,7395_2.txt,5043_3.txt,4990_1.txt,7261_3.txt,4726_1.txt,5489_1.txt,4031_4.txt,5892_2.txt,8965_1.txt,8198_4.txt,...,11127_3.txt,10073_4.txt,291_3.txt,1963_4.txt,11379_3.txt,11580_2.txt,7514_1.txt,9739_4.txt,5129_1.txt,3587_3.txt
abandoned,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ability,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abnormal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aboard,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ziyi,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zombie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zombierelated,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
total_freq = freq_df.sum(axis=1)
all_metrics_df = pd.DataFrame({"total_tf": total_freq})

all_metrics_df["df"] = document_frequency#df_dict.values()
all_metrics_df['idf'] = np.log10(100 / all_metrics_df['df'])
all_metrics_df['tf_idf'] = all_metrics_df['idf'] * all_metrics_df['total_tf']
all_metrics_df_sorted = all_metrics_df.sort_values(by='total_tf', ascending=False)
normalized_column = normalize(all_metrics_df_sorted[['tf_idf']], norm='l2', axis=0)
all_metrics_df_sorted['normalized_tf_idf'] = normalized_column

all_metrics_df_sorted.head(10)

,total_tf,df,idf,tf_idf,normalized_tf_idf
movie,6.442085,68,0.167491,1.078992,0.046375
film,5.827881,58,0.236572,1.378713,0.059258
like,3.100557,52,0.283997,0.880548,0.037846
one,3.064812,54,0.267606,0.820163,0.035251
character,3.019550,39,0.408935,1.234801,0.053072
get,2.517052,35,0.455932,1.147604,0.049324
bad,2.459905,31,0.508638,1.251202,0.053777
really,2.458946,36,0.443697,1.091028,0.046893
plot,2.419400,37,0.431798,1.044693,0.044901
would,2.418317,36,0.443697,1.073001,0.046118


In [32]:
all_metrics_df_new_sorted = all_metrics_df_sorted.sort_values(by='tf_idf', ascending=False)
all_metrics_df_new_sorted.head(10)

,total_tf,df,idf,tf_idf,normalized_tf_idf
actor,1.832096,15,0.823909,1.509480,0.064878
kibbutz,0.719361,1,2.000000,1.438723,0.061837
show,1.658870,14,0.853872,1.416462,0.060880
film,5.827881,58,0.236572,1.378713,0.059258
story,1.999280,21,0.677781,1.355073,0.058241
joke,1.199305,8,1.096910,1.315529,0.056542
pretty,1.459821,13,0.886057,1.293484,0.055594
book,1.204717,9,1.045757,1.259842,0.054148
bad,2.459905,31,0.508638,1.251202,0.053777
horror,1.081837,7,1.154902,1.249416,0.053700


In [23]:
word_freq_dicts = {}

for random_files_index, random_file in enumerate(random_files):
    name_doc = "Doc"+ str(random_files_index) + "tf_idf"
    word_freq_per_doc_dict = {}
    for word_index, word in enumerate(feature_names):
        word_freq_per_doc_dict[word] = tfidf_vectors[random_files_index][word_index]
    
    word_freq_dicts[random_file] = word_freq_per_doc_dict

freq_df = pd.DataFrame(word_freq_dicts)
freq_df

,7395_2.txt,5043_3.txt,4990_1.txt,7261_3.txt,4726_1.txt,5489_1.txt,4031_4.txt,5892_2.txt,8965_1.txt,8198_4.txt,...,11127_3.txt,10073_4.txt,291_3.txt,1963_4.txt,11379_3.txt,11580_2.txt,7514_1.txt,9739_4.txt,5129_1.txt,3587_3.txt
abandoned,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ability,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
able,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abnormal,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aboard,0.0,0.000000,0.0,0.0,0.0,0.0,0.051401,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ziyi,0.0,0.228334,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zombie,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zombierelated,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zone,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# Preprocess the query_text (replace this with your actual query)
query_text = "made me cry"

# Calculate the TF-IDF vector for the query
query_vector = tfidf_vectorizer.transform([query_text]).toarray()

from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between the query vector and document vectors
similarities = cosine_similarity(query_vector, tfidf_vectors)

similarities

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.00922838, 0.02552782, 0.02687279, 0.        ,
        0.03621537, 0.        , 0.04344503, 0.        , 0.01999301,
        0.        , 0.01997679, 0.04441722, 0.        , 0.06545443,
        0.        , 0.02356506, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.01807174, 0.        , 0.01322742,
        0.00759715, 0.03404838, 0.        , 0.        , 0.        ,
        0.        , 0.00795893, 0.        , 0.        , 0.        ,
        0.02459195, 0.02343485, 0.03507767, 0.0205157 , 0.05254071,
        0.01936039, 0.        , 0.        , 0.        , 0.        ,
        0.01146033, 0.        , 0.        , 0.        , 0.01371885,
        0.        , 0.        , 0.        , 0.05334392, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.02

In [42]:
# Get the indices of the top-N most similar documents
top_indices = np.argsort(similarities[0])[::-1][:5]

top_titles = []
top_file_names = []

for index in top_indices:
    print("Title: ",titles[index], end=" ---- ")
    print("File name: ", random_files[index], end=" ---- ")
    print("Review: ", documents[index])

Title:  Freddy's Dead: The Final Nightmare (1991)  ---- File name:  2373_3.txt ---- Review:  warning review contains mild spoilersa couple year back managed see first five film franchise planning overview whole elm st series however two year find cant remember enough order guess couldnt made much impression recall sequel dream warrior particular werent bad often made though even original classic generally predictability premise people fall asleep get murdered dream doesnt lend narrative tension recall much first five film know never plumbed depth freddys deadan indication sick freddy public point judged fact film promoted solely character demise fact movie conclusion even hidden fact entire purpose film go illustrate vacant soulless cynical venture wastaking morally questionable idea child molester charismatic villain robert englunds innowayscary interpretation boom laughter always thought freddys mockery teenage victim le aimed character teenage audience could ever watch tripe like en

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
similarity_df = pd.DataFrame(cosine_similarities)
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.080066,0.061542,0.058908,0.052581,0.060854,0.051660,0.043490,0.089278,0.068814,...,0.069883,0.028730,0.062394,0.052185,0.052840,0.054693,0.058060,0.039148,0.039159,0.041565
1,0.080066,1.000000,0.104372,0.096369,0.035761,0.039454,0.074098,0.094881,0.028813,0.063958,...,0.044616,0.059069,0.077800,0.061235,0.019349,0.059608,0.021587,0.055069,0.011562,0.021278
2,0.061542,0.104372,1.000000,0.046375,0.053976,0.030418,0.065979,0.085059,0.012954,0.037902,...,0.013196,0.071385,0.044981,0.003269,0.030303,0.023985,0.121208,0.050611,0.007815,0.020025
3,0.058908,0.096369,0.046375,1.000000,0.035663,0.050514,0.069387,0.082785,0.071459,0.036879,...,0.024534,0.053544,0.036607,0.034040,0.027670,0.025298,0.022265,0.039101,0.011478,0.022963
4,0.052581,0.035761,0.053976,0.035663,1.000000,0.038615,0.057243,0.042069,0.039807,0.038635,...,0.023546,0.022473,0.031935,0.029203,0.040399,0.069555,0.024194,0.044359,0.013811,0.019603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.054693,0.059608,0.023985,0.025298,0.069555,0.025346,0.054146,0.022813,0.030327,0.029828,...,0.044763,0.024386,0.055134,0.046696,0.030511,1.000000,0.015322,0.026704,0.025253,0.027652
96,0.058060,0.021587,0.121208,0.022265,0.024194,0.037258,0.051903,0.026425,0.043188,0.052317,...,0.016540,0.057773,0.024460,0.042650,0.044446,0.015322,1.000000,0.060740,0.019970,0.010824
97,0.039148,0.055069,0.050611,0.039101,0.044359,0.031924,0.028422,0.047168,0.091028,0.048559,...,0.022713,0.025653,0.031669,0.036193,0.032029,0.026704,0.060740,1.000000,0.008955,0.005203
98,0.039159,0.011562,0.007815,0.011478,0.013811,0.023125,0.005583,0.037978,0.030906,0.035077,...,0.031969,0.029121,0.081960,0.038871,0.029176,0.025253,0.019970,0.008955,1.000000,0.020047


In [43]:
threshold = 0.1  # Ajusta el umbral según tu criterio

similar_pairs = []
for i in range(len(documents)):
    for j in range(i+1, len(documents)):
        if cosine_similarities[i][j] > threshold:
            similar_pairs.append((titles[i], titles[j], cosine_similarities[i][j]))

In [46]:
result_df = pd.DataFrame(similar_pairs, columns=["title1", "title2", "cosine"])
new_result_df = result_df.sort_values(by='cosine', ascending=False)

new_result_df.head(10)

,title1,title2,cosine
92,The Witches Hammer (2006),The Midnight Meat Train (2008),0.196291
124,Prizzi's Honor (1985),Rånarna (2003),0.182096
62,Universal Soldier: The Return (1999),The Midnight Meat Train (2008),0.177575
61,Universal Soldier: The Return (1999),Love.com (2002),0.176448
60,Universal Soldier: The Return (1999),The Witches Hammer (2006),0.176369
126,Love.com (2002),The Midnight Meat Train (2008),0.168070
63,Second to Die (2002),Ekstase (1933),0.167074
58,Ricordati di me (2003),Tigerland (2000),0.157326
59,Universal Soldier: The Return (1999),United (2003),0.154534
88,The Witches Hammer (2006),Love.com (2002),0.153113
